In [63]:
from numpy import *
import re
import operator
import feedparser

In [2]:
def loadDataSet():
    postingList = [['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
                 ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
                 ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
                 ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
                 ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
                 ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']]
    classVec = [0, 1, 0, 1, 0, 1]
    return postingList, classVec
    

In [3]:
def createVocabList(dataSet):
    vocabSet = set([])
    for document in dataSet:
        vocabSet = vocabSet | set(document)
    return list(vocabSet)

In [4]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print(f'the word {word} is not in my vocabulary!')
    return returnVec

In [5]:
def trainNB0(trainMatix, trainCategory):
    numTrainDocs = len(trainMatix)
    numWords = len(trainMatix[0])
    pAbusive = sum(trainCategory)/float(numTrainDocs)
    p0Num = ones(numWords)
    p1Num = ones(numWords)
    p0Deom = 2.0
    p1Denom = 2.0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:
            p1Num += trainMatix[i]
            p1Denom += sum(trainMatix[i])
        else:
            p0Num += trainMatix[i]
            p0Deom += sum(trainMatix[i])
    p1Vect = log(p1Num/p1Denom)
    p0Vect = log(p0Num/p0Deom)
    return p0Vect, p1Vect, pAbusive

In [6]:
list0Posts, listClasses = loadDataSet()

In [7]:
myVocabList = createVocabList(list0Posts)

In [8]:
myVocabList

['so',
 'to',
 'posting',
 'him',
 'please',
 'stop',
 'dog',
 'mr',
 'cute',
 'food',
 'steak',
 'park',
 'take',
 'how',
 'maybe',
 'my',
 'not',
 'flea',
 'I',
 'garbage',
 'has',
 'buying',
 'dalmation',
 'ate',
 'is',
 'quit',
 'problems',
 'licks',
 'help',
 'love',
 'worthless',
 'stupid']

In [9]:
trainMat = []
for postindoc in list0Posts:
    trainMat.append(setOfWords2Vec(myVocabList, postindoc))


In [10]:
p0V, p1V, pAb = trainNB0(trainMat, listClasses)

In [11]:
pAb

0.5

In [12]:
p0V

array([-2.56494936, -2.56494936, -3.25809654, -2.15948425, -2.56494936,
       -2.56494936, -2.56494936, -2.56494936, -2.56494936, -3.25809654,
       -2.56494936, -3.25809654, -3.25809654, -2.56494936, -3.25809654,
       -1.87180218, -3.25809654, -2.56494936, -2.56494936, -3.25809654,
       -2.56494936, -3.25809654, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -2.56494936, -2.56494936, -2.56494936, -2.56494936,
       -3.25809654, -3.25809654])

In [13]:
p1V

array([-3.04452244, -2.35137526, -2.35137526, -2.35137526, -3.04452244,
       -2.35137526, -1.94591015, -3.04452244, -3.04452244, -2.35137526,
       -3.04452244, -2.35137526, -2.35137526, -3.04452244, -2.35137526,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -2.35137526,
       -3.04452244, -2.35137526, -3.04452244, -3.04452244, -3.04452244,
       -2.35137526, -3.04452244, -3.04452244, -3.04452244, -3.04452244,
       -1.94591015, -1.65822808])

In [14]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify*p1Vec) + log(pClass1)
    p0 = sum(vec2Classify*p0Vec) + log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0
    

In [15]:
def testingNB():
    list0Posts, listClasses = loadDataSet()
    myVocabList = createVocabList(list0Posts)
    trainMat = []
    for postindoc in list0Posts:
        trainMat.append(setOfWords2Vec(myVocabList, postindoc))
    p0V, p1V, pAb = trainNB0(array(trainMat), array(listClasses))
    testEntry = ['love', 'my', 'dalmation']
    thiSDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(f'{testEntry}, classified as {classifyNB(thiSDoc, p0V, p1V, pAb)}')
    testEntry = ['stupid', 'garbage']
    thiSDoc = array(setOfWords2Vec(myVocabList, testEntry))
    print(f'{testEntry}, classified as {classifyNB(thiSDoc, p0V, p1V, pAb)}')

In [16]:
testingNB()

['love', 'my', 'dalmation'], classified as 0
['stupid', 'garbage'], classified as 1


In [17]:
def bagOfWords2Vec(vocabList, inputSet):
    returnVec = [0]*len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList[word]] +=1
    return returnVec

In [26]:
mySent = 'this book is the best book on python or ML. I have ever laid eyes upon.'
mySent.split()

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'ML.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [27]:
regEx = re.compile('\\W+')
listOfTokenes = regEx.split(mySent)
listOfTokenes

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'ML',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon',
 '']

In [28]:
[tok for tok in listOfTokenes if len(tok)>0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'ML',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [30]:
[tok.lower() for tok in listOfTokenes if len(tok) > 0]

['this',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'python',
 'or',
 'ml',
 'i',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon']

In [32]:
emailText = open('email/ham/6.txt').read()
listOfTokenes = regEx.split(emailText)

In [33]:
listOfTokenes

['Hello',
 'Since',
 'you',
 'are',
 'an',
 'owner',
 'of',
 'at',
 'least',
 'one',
 'Google',
 'Groups',
 'group',
 'that',
 'uses',
 'the',
 'customized',
 'welcome',
 'message',
 'pages',
 'or',
 'files',
 'we',
 'are',
 'writing',
 'to',
 'inform',
 'you',
 'that',
 'we',
 'will',
 'no',
 'longer',
 'be',
 'supporting',
 'these',
 'features',
 'starting',
 'February',
 '2011',
 'We',
 'made',
 'this',
 'decision',
 'so',
 'that',
 'we',
 'can',
 'focus',
 'on',
 'improving',
 'the',
 'core',
 'functionalities',
 'of',
 'Google',
 'Groups',
 'mailing',
 'lists',
 'and',
 'forum',
 'discussions',
 'Instead',
 'of',
 'these',
 'features',
 'we',
 'encourage',
 'you',
 'to',
 'use',
 'products',
 'that',
 'are',
 'designed',
 'specifically',
 'for',
 'file',
 'storage',
 'and',
 'page',
 'creation',
 'such',
 'as',
 'Google',
 'Docs',
 'and',
 'Google',
 'Sites',
 'For',
 'example',
 'you',
 'can',
 'easily',
 'create',
 'your',
 'pages',
 'on',
 'Google',
 'Sites',
 'and',
 'share',


In [34]:
def textParse(bigString):
    listOfTokenes = re.split(r'\W+', bigString)
    return [tok.lower() for tok in listOfTokenes if len(tok) > 2]


In [40]:
def spamTest():
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):
        wordList = textParse(open('email/spam/%d.txt' %i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1)
        wordList = textParse(open('email/ham/%d.txt' %i).read())
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)
    trainingSet = list(range(50))
    testSet = []
    for i in range(10):
        randIndex = int(random.uniform(0, len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])
    trainMat = []
    trainClasses = []
    for docIndex in trainingSet:
        trainMat.append(setOfWords2Vec(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V, p1V, pSpam = trainNB0(array(trainMat), array(trainClasses))
    errorCount = 0
    for docIndex in testSet:
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])
        if classifyNB(array(wordVector), p0V, p1V, pSpam) != classList[docIndex]:
            errorCount +=1
    print(f'Error rate is {float(errorCount/len(testSet))}')

In [53]:
spamTest()

Error rate is 0.1


In [66]:
import requests

headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
response = requests.get('http://newyork.craigslist.org/stp/index.rss', headers=headers)
ny = feedparser.parse(response.content)

In [67]:
ny['entries']
len(ny['entries'])

0

In [65]:
def calcMostFreq(vocabList, fullText):
    freqDict = {}
    for token in vocabList:
        freqDict[token] = fullText.Count(token)
    sortedFreq = sorted(freqDict.items(), key=operator.itemgetter(1), reverse=True)
    return sortedFreq[:30]

In [69]:
def localWords(feed1,feed0):
    import feedparser
    docList=[]; classList = []; fullText =[]
    minLen = min(len(feed1['entries']),len(feed0['entries']))
    for i in range(minLen):
        wordList = textParse(feed1['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(1) #NY is class 1
        wordList = textParse(feed0['entries'][i]['summary'])
        docList.append(wordList)
        fullText.extend(wordList)
        classList.append(0)
    vocabList = createVocabList(docList)#create vocabulary
    top30Words = calcMostFreq(vocabList,fullText)   #remove top 30 words
    for pairW in top30Words:
        if pairW[0] in vocabList: vocabList.remove(pairW[0])
    trainingSet = range(2*minLen); testSet=[]           #create test set
    for i in range(20):
        randIndex = int(random.uniform(0,len(trainingSet)))
        testSet.append(trainingSet[randIndex])
        del(trainingSet[randIndex])  
    trainMat=[]; trainClasses = []
    for docIndex in trainingSet:#train the classifier (get probs) trainNB0
        trainMat.append(bagOfWords2VecMN(vocabList, docList[docIndex]))
        trainClasses.append(classList[docIndex])
    p0V,p1V,pSpam = trainNB0(array(trainMat),array(trainClasses))
    errorCount = 0
    for docIndex in testSet:        #classify the remaining items
        wordVector = bagOfWords2VecMN(vocabList, docList[docIndex])
        if classifyNB(array(wordVector),p0V,p1V,pSpam) != classList[docIndex]:
            errorCount += 1
    print('the error rate is: ',float(errorCount)/len(testSet))
    return vocabList,p0V,p1V

def getTopWords(ny,sf):
    import operator
    vocabList,p0V,p1V=localWords(ny,sf)
    topNY=[]; topSF=[]
    for i in range(len(p0V)):
        if p0V[i] > -6.0 : topSF.append((vocabList[i],p0V[i]))
        if p1V[i] > -6.0 : topNY.append((vocabList[i],p1V[i]))
    sortedSF = sorted(topSF, key=lambda pair: pair[1], reverse=True)
    print( "SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**SF**")
    for item in sortedSF:
        print(item[0])
    sortedNY = sorted(topNY, key=lambda pair: pair[1], reverse=True)
    print("NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**NY**")
    for item in sortedNY:
        print(item[0])